In [6]:
from datetime import datetime, timedelta, timezone
import pytz
import sqlite3
import requests
from time import sleep
from tqdm import tqdm
import pandas as pd
import json

# Carregando as credenciais do arquivo externo
with open("credentials.json", "r") as cred_file:
    credentials = json.load(cred_file)

API_KEY = credentials["API_KEY"]
PROJECT_ID = credentials["PROJECT_ID"]
EMAIL = credentials["EMAIL"]
PASSWORD = credentials["PASSWORD"]
DATABASE_PATH = credentials["DATABASE_PATH"]
LOG_FILE = credentials["LOG_FILE"]

# Configuração inicial
START_TIME = "1 de agosto de 2024 às 03:00:00"
END_TIME = "24 de agosto de 2024 às 22:00:00"
LIMIT_DOCS = 100000
COLLECTION_NAME = "system-1"  # Nome da coleção a ser coletada
TIMESTAMP_FIELD = "Timestamp"  # Campo que contém o timestamp

AUTH_URL = f"https://identitytoolkit.googleapis.com/v1/accounts:signInWithPassword?key={API_KEY}"
FIRESTORE_URL = f"https://firestore.googleapis.com/v1/projects/{PROJECT_ID}/databases/(default)/documents:runQuery"


In [2]:
def format_custom_timestamp(date_str):
    """Converte string de data para ISO 8601 com Z."""
    months = {
        "janeiro": 1, "fevereiro": 2, "março": 3, "abril": 4, "maio": 5,
        "junho": 6, "julho": 7, "agosto": 8, "setembro": 9, "outubro": 10,
        "novembro": 11, "dezembro": 12
    }
    parts = date_str.split(" às ")
    date_part = parts[0]
    time_part = parts[1]

    day, month_str, year = date_part.split(" de ")
    month = months[month_str.strip()]
    hour, minute, second = map(int, time_part.split(":"))

    local_dt = datetime(year=int(year), month=month, day=int(day), hour=hour, minute=minute, second=second)
    tz = pytz.timezone("America/Sao_Paulo")
    local_dt = tz.localize(local_dt)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime("%Y-%m-%dT%H:%M:%SZ")

def get_id_token():
    """Obtém o token de autenticação."""
    payload = {
        "email": EMAIL,
        "password": PASSWORD,
        "returnSecureToken": True
    }
    response = requests.post(AUTH_URL, json=payload)
    if response.status_code == 200:
        return response.json().get("idToken"), datetime.now(timezone.utc) + timedelta(hours=1)
    else:
        raise Exception(f"Erro na autenticação: {response.json()}")

def query_firestore_by_timestamp(id_token, collection_name, timestamp_field, start_time, end_time):
    """Consulta documentos no Firestore por intervalo de tempo."""
    headers = {"Authorization": f"Bearer {id_token}"}
    payload = {
        "structuredQuery": {
            "from": [{"collectionId": collection_name}],
            "where": {
                "compositeFilter": {
                    "op": "AND",
                    "filters": [
                        {
                            "fieldFilter": {
                                "field": {"fieldPath": timestamp_field},
                                "op": "GREATER_THAN_OR_EQUAL",
                                "value": {"timestampValue": start_time}
                            }
                        },
                        {
                            "fieldFilter": {
                                "field": {"fieldPath": timestamp_field},
                                "op": "LESS_THAN_OR_EQUAL",
                                "value": {"timestampValue": end_time}
                            }
                        }
                    ]
                }
            },
            "orderBy": [{"field": {"fieldPath": timestamp_field}, "direction": "ASCENDING"}]
        }
    }
    response = requests.post(FIRESTORE_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Erro ao consultar Firestore: {response.json()}")

def setup_database():
    """Configura o banco SQLite para armazenar os dados."""
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS firebase_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT
        )
    """)
    conn.commit()
    return conn

def add_column_if_not_exists(cursor, column_name):
    """Adiciona uma coluna no banco de dados, caso ela não exista."""
    cursor.execute(f"PRAGMA table_info(firebase_data)")
    existing_columns = [info[1] for info in cursor.fetchall()]
    if column_name not in existing_columns:
        cursor.execute(f"ALTER TABLE firebase_data ADD COLUMN {column_name} TEXT")

def save_to_database(conn, data):
    """Salva os dados no banco SQLite."""
    cursor = conn.cursor()
    for item in data:
        fields = item.get("document", {}).get("fields", {})
        if fields:
            for field_name, field_value in fields.items():
                add_column_if_not_exists(cursor, field_name)
            column_names = ", ".join(fields.keys())
            placeholders = ", ".join("?" for _ in fields.keys())
            values = [field_value.get(list(field_value.keys())[0]) for field_value in fields.values()]
            cursor.execute(f"INSERT INTO firebase_data ({column_names}) VALUES ({placeholders})", values)
    conn.commit()

def log_event(message):
    """Registra mensagens em um arquivo de log."""
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{datetime.now().isoformat()} - {message}\n")

In [3]:
def main():
    log_event("Início do backup.")
    start_time = format_custom_timestamp(START_TIME)
    end_time = format_custom_timestamp(END_TIME)
    conn = setup_database()
    token, token_expiry = get_id_token()

    total_hours = int((datetime.fromisoformat(end_time.replace("Z", "+00:00")) -
                       datetime.fromisoformat(start_time.replace("Z", "+00:00"))).total_seconds() / 3600)
    documents_count = 0

    with tqdm(total=total_hours, desc="Coletando dados") as pbar:
        current_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))
        while documents_count < LIMIT_DOCS:
            next_time = (current_time + timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
            if next_time > end_time:
                next_time = end_time

            if datetime.now(timezone.utc) > token_expiry:
                token, token_expiry = get_id_token()

            try:
                tqdm_desc = f"Coletando {current_time.strftime('%Y-%m-%dT%H:%M:%S')} até {next_time}"
                pbar.set_description(tqdm_desc)
                
                documents = query_firestore_by_timestamp(token, COLLECTION_NAME, TIMESTAMP_FIELD, current_time.strftime("%Y-%m-%dT%H:%M:%SZ"), next_time)
                save_to_database(conn, documents)
                documents_count += len(documents)
                if documents_count >= LIMIT_DOCS:
                    log_event(f"Backup interrompido: limite de {LIMIT_DOCS} documentos atingido.")
                    print(f"Limite de {LIMIT_DOCS} documentos atingido. Parando.")
                    break
            except Exception as e:
                log_event(f"Erro durante a consulta: {e}")
                print(f"Erro durante a consulta: {e}")
            
            current_time += timedelta(hours=1)
            pbar.update(1)
            sleep(2)

    conn.close()
    log_event(f"Término do backup. Total de documentos coletados: {documents_count}. Última hora coletada: {current_time}.")
    print("Backup finalizado.")

In [4]:
if __name__ == "__main__":
    main()

Coletando 2024-08-06 17:00:00 até 2024-08-06T18:00:00Z:  23%|█████▌                  | 131/571 [07:22<24:45,  3.38s/it]

Limite de 100000 documentos atingido. Parando.
Backup finalizado.
